# Goal-oriented self-improvement for Automated Theorem Proving

## Method
We are basically thinking of extending minimo by introducing a goal i.e. a set of problems the model should eventually solve. We do this by adding loss-term forcing the model to sample conjectures from our goal set. We control this ‘forcing’ using a hyperparameter `alpha`. A higher value gives the `progess_loss` more value, thus pushes the model towards the goal stronger.

## Experiments

For the now we use a very simple goal set. It consists of a single goal which is proving the theorem: 

`(1 + a0) + a1 = 1 + (a0 + a1)`

Or in peano:

`[('a0 : nat) -> ('a1 : nat) -> (= (+ (s 'a0) 'a1) (s (+ 'a0 'a1)))]`

### 1. Overfit on the goal set
After implementing the goal-conditioning, let's run experiments with `alpha=0` and `alpha=1`. We expect the `progress_loss` to go down i.e. the model overfits to the `final_goal` set. However, the actual `train_loss` shouldn’t go down. 

In [1]:
# start a job with alpha=0 as a baseline
!sbatch --job-name=train_alpha_0 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0"

# start a job with alpha=1 to overfit on the goals
!sbatch --job-name=train_alpha_1 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1"

Submitted batch job 1274470
Submitted batch job 1274471


Works! The ``training_loss`` diverges for `alpha=1`. The `progress_loss` struggles to go below a certain threshold for `alpha=0`. Interestingly, the former overfits to sampling the final theorem and can't solve any conjectures in the second iteration. We need something smarter for alpha.

### 2. Try different values for alpha

We try different schedules for alpha in the hope that the training loss and progress loss both converge. 

In [2]:
!sbatch --job-name=train_alpha_0_8 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.8"

!sbatch --job-name=train_alpha_0_6 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.6"

!sbatch --job-name=train_alpha_0_4 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.4"

!sbatch --job-name=train_alpha_0_2 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.2"

Submitted batch job 1274473
Submitted batch job 1274474
Submitted batch job 1274475
Submitted batch job 1274476


This works as well. We also observe a correlation between alpha values and the ratio of conjectures that the model is able to prove per iteration. The higher the alpha value, the fewer problems the model can solve. What if we could let the model explore problems for itself for a while and at a later stage push it towards our goal?

### 3 Alpha Schedules

We implement fancy schedules to 'warm-up' alpha over several iterations. The options are 

`alpha_schedule = [ constant | linear | quadratic | cubic ]`

#### 3.1 Warm up alpha to 1.0

As a first step we warm up alpha to 1.0. 

In [ ]:
# !sbatch --job-name=train_alpha_lin --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=linear"

!sbatch --job-name=train_alpha_quad --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=quadratic"

!sbatch --job-name=train_alpha_cubic --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=cubic"